In [23]:
import importlib

import numpy as np
import matplotlib.pyplot as plt

import hw5_version
importlib.reload(hw5_version)




<module 'hw5_version' from 'C:\\Users\\George\\Desktop\\CS155_Project2\\CS155_PROJECT2\\Visualizations\\hw5_version.py'>

In [24]:
Y_train = np.loadtxt('./data/train.txt').astype(int)
Y_test = np.loadtxt('./data/test.txt').astype(int)

M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies


K = 20
eta = 0.03
reg = 0


Using the Homework 5 version of SVD 

In [25]:
U,V, err = hw5_version.train_model(M, N, K, eta, reg, Y_train)

Usin